This is a notebook demonstrating the Transflower model from the paper [Transflower:Probabilistic autoregressive dance generation with multimodal attention](http://metagen.ai/transflower)

Transflower combines transformers with autoregressive normalizing flows, to do sequence modelling of high dimensional continuous signals. Here we apply it to dance generation, where we condition the generated motion on a piece of music.

To run this notebook simply run each cell in order, either by pressing ctrl+enter or by clicking the little play icons.


In [1]:
import os
from os import mkdir, path
def cd(dir):
  os.chdir(dir)
transflower_folder_name = './' #@param {type:"string"}
#working_dir=!pwd
#working_dir=working_dir[0]

In [2]:
#@title Select which models to download (in the next step)
download_Transflower = True #@param {type:"boolean"}
download_Transflower_finetune = True #@param {type:"boolean"}
download_MoGlow = True #@param {type:"boolean"}
download_Transformer = True #@param {type:"boolean"}
download_Transflower_2nd_checkpoint = False #@param {type:"boolean"}

import os

os.environ["download_Transflower"] = str(download_Transflower)
os.environ["download_Transflower_finetune"] = str(download_Transflower_finetune)
os.environ["download_MoGlow"] = str(download_MoGlow)
os.environ["download_Transformer"] = str(download_Transformer)
os.environ["download_Transflower_2nd_checkpoint"] = str(download_Transflower_2nd_checkpoint)

In [ ]:
#@title Prepare transflower and dependencies
#@markdown Run this cell to download transflower to your drive (only will if it hasn't downloaded already), and prepare dependencies
#%%capture
%%bash
if [[ -z "$(ls -A .)" ]]; then
  git clone https://github.com/guillefix/transflower-lightning .
  mkdir songs
  mkdir training/experiments
  if [ "$download_Transflower" == "True" ]; then
  gsutil -m cp -r gs://metagen/models/transflower_expmap_old training/experiments/
  fi
  if [ "$download_Transflower_finetune" == "True" ]; then
  gsutil -m cp -r gs://metagen/models/transflower_expmap_finetune2_old training/experiments/
  fi
  if [ "$download_Transflower_2nd_checkpoint" == "True" ]; then
  gsutil -m cp -r gs://metagen/models/transflower_expmap training/experiments/
  fi
  if [ "$download_Transformer" == "True" ]; then
  gsutil -m cp -r gs://metagen/models/transformer_expmap1 training/experiments/
  fi
  if [ "$download_MoGlow" == "True" ]; then
  gsutil -m cp -r gs://metagen/models/moglow_expmap training/experiments/
  fi
  gsutil -m cp -r gs://metagen/scalers/* songs/
  gsutil -m cp -r gs://metagen/seeds/* songs/
fi
pip install -r requirements.txt
pip install mido madgrad
apt-get install ffmpeg
sudo curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl
sudo chmod a+rx /usr/local/bin/youtube-dl

In [4]:
#@title Get song
#@markdown Choose to download song from youtube, otherwise it asks you to upload file
download_from_youtube = True #@param {type:"boolean"}
youtube_url = 'https://www.youtube.com/watch?v=mE8YOyxJ3x4' #@param {type:"string"}
youtube_url = "https://www.youtube.com/watch?v=0rUmLs3hul8"
if download_from_youtube:
  !export youtube_url
  !youtube-dl -x --audio-format wav --restrict-filenames $youtube_url
  filename=!youtube-dl -x --audio-format mp3 --get-filename --restrict-filenames $youtube_url
  filename = os.path.splitext(filename[0])[0]+".wav"
  filename_new = filename[:50].replace(".","_")+".wav"
  !mv $filename songs/$filename_new
  filename = filename_new
  filename=filename[:-4] # removing final extensions
else:
  res=files.upload()
  filename=list(res.keys())[0]
  os.rename(filename,"songs/"+filename)
  filename=filename[:-4] # removing final extensions

print(filename)

[youtube] 0rUmLs3hul8: Downloading webpage
[download] Destination: _-0rUmLs3hul8.webm
[download] 100% of 3.65MiB in 00:5127KiB/s ETA 00:00
[ffmpeg] Destination: _-0rUmLs3hul8.wav
Deleting original file _-0rUmLs3hul8.webm (pass -k to keep)
_-0rUmLs3hul8_wav


In [5]:
#@title Extract music features
#@markdown This will extract the features of the downloaded song(s)
#%%capture
#%%bash
!chmod +x ./feature_extraction/audio_feature_extraction_test.sh
!chmod +x ./feature_extraction/script_to_list_filenames
!./feature_extraction/audio_feature_extraction_test.sh songs/

hwloc/linux: Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).
hwloc/linux: Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).
0
creating multi_mel of size 80
creating feature file 0
HIIIIIIOOO
HOOOOOO
HOOOOOO
/home/kitsume/Desktop/projects/motion/transflower-memo/.venv/lib/python3.9/site-packages/pl_bolts/callbacks/data_monitor.py:20: UnderReviewWarning: The feature warn_missing_pkg is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  warn_missing_pkg("wandb")
/home/kitsume/Desktop/p

In [9]:
#@title Generate dance 
#@markdown Generate dance ^^ 💃

#@markdown If it shows some nans ignore them <small>(its some issue if motion seed clip is shorter than the song)</small>

#@markdown You can choose one of the following motion seeds, that will affect the style (you can also input a custom one, but that's more advanced)
motion_seed = 'Casual2' #@param ["Free style", "Casual", "Hip Hop", "Break dance", "Groovenet", "Casual2"]

#@markdown You can choose among these two models. If you want to try the baselines MoGlow or Transformer from the paper, you'd need to have downloaded them in the above step
model = 'Transflower fine tuned' #@param ["Transflower", "Transflower fine tuned", "Transformer", "MoGlow", "Transflower_second_checkpoint"]
# model = 'Transflower' #@param ["Transflower", "Transflower fine tuned"] {allow-input: true}
generate_bvh = True #@param {type:"boolean"}
generate_video = True #@param {type:"boolean"}
short_test_run = False #@param {type:"boolean"}
seed_options = ["Free style", "Casual", "Hip Hop", "Break dance", "Groovenet", "Casual2"]
seed = str(seed_options.index(motion_seed) + 1)
!chmod +x ./script_generate.sh
!cp 'songs/seed'{seed}'.npy' 'songs/'{filename}'.expmap_scaled_20.npy'

if model == "Transflower":
  model_string = "transflower_expmap_old"
elif model == "Transflower fine tuned":
  model_string = "transflower_expmap_finetune2_old"
elif model == "MoGlow":
  model_string = "moglow_expmap"
elif model == "Transformer":
  model_string = "transformer_expmap1"
elif model == "Transflower_second_checkpoint":
  model_string == "transflower_expmap"
else:
  raise NotImplementedError("model "+model+" not implemented!")

extra_args = ""
if generate_bvh:
  extra_args += "--generate_bvh "
if generate_video:
  extra_args += "--generate_video "
if short_test_run:
  extra_args += "--max_length=512 "

!./script_generate.sh {model_string} {filename} {extra_args} --data_dir=songs

cp: 'songs/seed{seed}.npy' を stat できません: そのようなファイルやディレクトリはありません
{model_string} {filename}
mkdir: ディレクトリ `inference/generated/' を作成できません: ファイルが存在します
mkdir: ディレクトリ `inference/generated/{model_string}' を作成できません: ファイルが存在します
mkdir: ディレクトリ `inference/generated/{model_string}/predicted_mods' を作成できません: ファイルが存在します
mkdir: ディレクトリ `inference/generated/{model_string}/videos' を作成できません: ファイルが存在します
HIIIIIIOOO
HOOOOOO
HOOOOOO
/home/kitsume/Desktop/projects/motion/transflower-memo/.venv/lib/python3.9/site-packages/pl_bolts/callbacks/data_monitor.py:20: UnderReviewWarning: The feature warn_missing_pkg is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  warn_missing_pkg("wandb")
/home/kitsume/Desktop/projects/motion/transflower-memo/.venv/lib/python3.9/site-packages/pl_bolts/m

The generated video (and bvh file if generated) are found inside `transflower_code/inference/generated/[model_name]/videos`

In [10]:
#@title Visualize
#@markdown This will copy the generated dance to a server and open a visualizer
# %%capture
import uuid
from IPython.display import Javascript
hash = str(uuid.uuid1())
!echo $(echo {filename})_{hash}
!cp inference/generated/{model_string}/videos/{filename}.bvh inference/generated/{model_string}/videos/{filename}_{hash}.bvh
!gsutil cp inference/generated/{model_string}/videos/{filename}_{hash}.bvh gs://metagendance/
!cp inference/generated/{model_string}/videos/{filename}.mp4.mp3 inference/generated/{model_string}/videos/{filename}_{hash}.bvh.mp3
!gsutil cp inference/generated/{model_string}/videos/{filename}_{hash}.bvh.mp3 gs://metagendance/
url = 'https://guillefix.github.io/bvh_visualizer/?name='+filename+'_'+hash
print("Opening "+url)
display(Javascript('window.open("{url}");'.format(url=url)))

_-0rUmLs3hul8_wav_aa3a62f4-8a8b-11ed-8bb1-25479b99edc0
Copying file://inference/generated/transflower_expmap_old/videos/_-0rUmLs3hul8_wav_aa3a62f4-8a8b-11ed-8bb1-25479b99edc0.bvh [Content-Type=application/octet-stream]...
- [1 files][  8.3 MiB/  8.3 MiB]                                                
Operation completed over 1 objects/8.3 MiB.                                      
Copying file://inference/generated/transflower_expmap_old/videos/_-0rUmLs3hul8_wav_aa3a62f4-8a8b-11ed-8bb1-25479b99edc0.bvh.mp3 [Content-Type=audio/mpeg]...
- [1 files][  3.5 MiB/  3.5 MiB]                                                
Operation completed over 1 objects/3.5 MiB.                                      
Opening https://guillefix.github.io/bvh_visualizer/?name=_-0rUmLs3hul8_wav_aa3a62f4-8a8b-11ed-8bb1-25479b99edc0


<IPython.core.display.Javascript object>

In [ ]:
# !cd ..; rm -r transflower_code